In [ ]:
###############################Meijering ridges + slice-wise local thresholds####################################

In [ ]:
# %%
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, meijering, threshold_local
import napari
import pandas as pd
import imageio
import cv2

from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, skeletonize_3d
)

# --------------------------
# Helpers
# --------------------------
def safe_minmax_norm(volume):
    vol = volume.astype(np.float32)
    vmin, vmax = float(vol.min()), float(vol.max())
    if vmax > vmin:
        vol = (vol - vmin) / (vmax - vmin)
    else:
        vol[:] = 0.0
    return vol

def to_uint8_stack(volume):
    vol = safe_minmax_norm(volume)
    return (vol * 255).astype(np.uint8)

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs axon channel (as in your original)
image = img_ch1        # lysosome channel
image_2 = img_ch2      # axon/soma channel

# --------------------------
# 2) SMOOTHING (CH1)
# --------------------------
image_smooth = gaussian(image, sigma=1)

# --------------------------
# 3) BLOB DETECTION (CH1)
# --------------------------
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
# convert sigma->radius for LoG blobs (3D)
blobs[:, 3] = blobs[:, 3] * np.sqrt(3)

print(f"Detected {len(blobs)} lysosomes.")

# --------------------------
# 4) REGIONAL QUANTIFICATION (CH1) on a 3D GRID
# --------------------------
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

z_idx = np.digitize(blobs[:, 0], z_bins) - 1
y_idx = np.digitize(blobs[:, 1], y_bins) - 1
x_idx = np.digitize(blobs[:, 2], x_bins) - 1

region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx

# --- Calculate unique ID and volume for each blob ---
blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3)

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0],
    "y": blobs[:, 1],
    "x": blobs[:, 2],
    "diameter_voxels": blobs[:, 3] * 2,
    "radius_voxels": blobs[:, 3],
    "volume_voxels": volumes,
    "region_id": region_labels
})

region_summary = df.groupby("region_id").agg(
    count=("radius_voxels", "size"),
    avg_diameter=("diameter_voxels", "mean"),
    total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
).reset_index()

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 5) NAPARI VISUALIZATION (CH1)
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image, name='Lysosome Channel')

# Color blobs by region for visual grouping
properties = {'region': region_labels.astype(int)}
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties=properties,
    face_color='region',
    face_colormap='tab20',
    name='Detected Lysosomes'
)

# ----Add region ID text labels at region centers (for CH1 grid)----
region_centers = []
region_ids = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)
region_ids = np.array(region_ids)

viewer.add_points(
    region_centers,
    size=8,
    face_color='black',
    name='Region Labels',
    text=[str(r) for r in region_ids]
)

# --------------------------
# 3B) AXON SEGMENTATION — OPTION 2 (Meijering + local thresholds)
# --------------------------
# Normalize CH2 into [0,1] and denoise lightly
ch2 = rescale_intensity(image_2.astype(np.float32), in_range='image', out_range=(0,1))
ch2 = gaussian(ch2, sigma=0.6, preserve_range=True)

# Meijering (bright ridges → black_ridges=False)
resp = meijering(ch2, sigmas=np.linspace(0.6, 2.0, 8), black_ridges=False)

# Local threshold per z-slice (handles depth-varying background)
axon_mask = np.zeros_like(resp, dtype=bool)
for z in range(resp.shape[0]):
    R = resp[z]
    # block_size must be odd; tweak offset factor (e.g., -0.3 to -0.7)*std for your data
    t = threshold_local(R, block_size=31, offset=-0.5*np.std(R))#51#0.5,#11#0.3
    axon_mask[z] = R > t

# Cleanup (remove speckles; open/close small gaps)
axon_mask = remove_small_objects(axon_mask, min_size=6000, connectivity=5)#400#4000####3
axon_mask = binary_opening(axon_mask, ball(2))#1
axon_mask = binary_closing(axon_mask, ball(2))#1

# (Optional) SKELETON for geometry/length
axon_skel = skeletonize_3d(axon_mask)

# --------------------------
# 4) REGIONAL QUANTIFICATION (CH2) on a 3D GRID
# --------------------------
num_bins_2 = (4, 4, 4)
z_bins_2 = np.linspace(0, image_2.shape[0], num_bins_2[0] + 1, dtype=int)
y_bins_2 = np.linspace(0, image_2.shape[1], num_bins_2[1] + 1, dtype=int)
x_bins_2 = np.linspace(0, image_2.shape[2], num_bins_2[2] + 1, dtype=int)

region_records = []
for zi in range(num_bins_2[0]):
    for yi in range(num_bins_2[1]):
        for xi in range(num_bins_2[2]):
            z0, z1 = z_bins_2[zi], z_bins_2[zi+1]
            y0, y1 = y_bins_2[yi], y_bins_2[yi+1]
            x0, x1 = x_bins_2[xi], x_bins_2[xi+1]
            rid = zi * (num_bins_2[1] * num_bins_2[2]) + yi * num_bins_2[2] + xi

            sub_mask  = axon_mask[z0:z1, y0:y1, x0:x1]
            sub_skel  = axon_skel[z0:z1, y0:y1, x0:x1]
            vox_total = sub_mask.size
            vox_axon  = int(sub_mask.sum())
            if vox_axon == 0:
                continue  # <-- skip this region entirely
            vox_skel  = int(sub_skel.sum())  # skeleton voxels ~ length proxy (in voxel units)

            region_records.append({
                "region_id": rid,
                "axon_voxels": vox_axon,
                #"region_voxels": vox_total,
                #"axon_volume_fraction": vox_axon / vox_total if vox_total > 0 else 0.0,
                #"skeleton_voxels": vox_skel
            })

df_axon_regions = pd.DataFrame(region_records)

# --------------------------
# 4B) COLOCALIZATION: ch1 blobs inside axon mask (CH2)
# --------------------------
def _in_bounds(z, y, x, shape):
    return (0 <= z < shape[0]) and (0 <= y < shape[1]) and (0 <= x < shape[2])

in_axon = []
for zc, yc, xc in blobs[:, :3]:
    zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
    if _in_bounds(zz, yy, xx, axon_mask.shape):
        in_axon.append(bool(axon_mask[zz, yy, xx]))
    else:
        in_axon.append(False)

df["in_axon_mask_ch2"] = in_axon

# per-region colocalization rates
coloc_summary = df.groupby("region_id").agg(
    blobs_total=("id", "size"),
    #blobs_in_axon=("in_axon_mask_ch2", "sum")
#).reset_index()
#coloc_summary["frac_blobs_in_axon"] = (
#    coloc_summary["blobs_in_axon"] / coloc_summary["blobs_total"].replace(0, np.nan)
)

# --------------------------
# 4C) SAVE CHANNEL-2 SUMMARIES
# --------------------------
df_axon_regions.to_csv("axon_regions_ch2.csv", index=False)
coloc_summary.to_csv("blobs_in_axon_by_region.csv", index=False)
print("Saved: axon_regions_ch2.csv and blobs_in_axon_by_region.csv")

# --------------------------
# 5B) NAPARI VISUALIZATION (CH2)
# --------------------------
viewer_2 = napari.Viewer()
viewer_2.add_image(image_2, name='Channel 2 (raw)')
viewer_2.add_labels(axon_mask.astype(np.uint8), name='Axon mask (Meijering + local)')
viewer_2.add_image(axon_skel.astype(np.uint8), name='Axon skeleton', blending='additive', contrast_limits=(0,1))

# --------------------------
# 6) MOVIES / OVERLAYS
# --------------------------
# CH1 video (raw)
img_norm = to_uint8_stack(image)
frames = [img_norm[z] for z in range(img_norm.shape[0])]

try:
    imageio.mimsave('lysosome_stack.mp4', frames, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack.mp4')
except TypeError:
    imageio.mimsave('lysosome_stack.gif', frames, fps=8)
    print('Saved: lysosome_stack.gif')

# CH1 video with blobs overlay
frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame = img_norm[z].copy()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    # Draw blobs at this z-slice
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
    for b in z_blobs:
        y, x = int(round(b[1])), int(round(b[2]))
        radius = int(round(b[3]))
        cv2.circle(frame_rgb, (x, y), radius, (0, 255, 255), 2)  # yellow
    frames_with_blobs.append(frame_rgb)

try:
    imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack_blobs.mp4')
except TypeError:
    imageio.mimsave('lysosome_stack_blobs.gif', frames_with_blobs, fps=8)
    print('Saved: lysosome_stack_blobs.gif')

# CH2 base stack (raw)
img_norm_2 = to_uint8_stack(image_2)
frames_2 = [img_norm_2[z2] for z2 in range(img_norm_2.shape[0])]
try:
    imageio.mimsave('axon_stack_ch2.mp4', frames_2, fps=8, format='FFMPEG')
    print("Saved: axon_stack_ch2.mp4")
except TypeError:
    imageio.mimsave('axon_stack_ch2.gif', frames_2, fps=8)
    print("Saved: axon_stack_ch2.gif")

# CH2 overlay with axon mask (colored)
frames_mask_overlay = []
for z2 in range(img_norm_2.shape[0]):
    base = cv2.cvtColor(img_norm_2[z2], cv2.COLOR_GRAY2BGR)
    mask = (axon_mask[z2].astype(np.uint8) * 255)
    color_mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(base, 1.0, color_mask, 0.35, 0.0)
    frames_mask_overlay.append(overlay)

try:
    imageio.mimsave('axon_mask_overlay_ch2.mp4', frames_mask_overlay, fps=8, format='FFMPEG')
    print("Saved: axon_mask_overlay_ch2.mp4")
except TypeError:
    imageio.mimsave('axon_mask_overlay_ch2.gif', frames_mask_overlay, fps=8)
    print("Saved: axon_mask_overlay_ch2.gif")

# --------------------------
# 7) RUN VIEWERS
# --------------------------
napari.run()

In [ ]:
#####VERSION 2#######################################################################################3

In [ ]:
# %%
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, meijering, threshold_local
import napari
import pandas as pd
import imageio
import cv2

from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, skeletonize_3d
)

# --------------------------
# Helpers
# --------------------------
def safe_minmax_norm(volume):
    vol = volume.astype(np.float32)
    vmin, vmax = float(vol.min()), float(vol.max())
    if vmax > vmin:
        vol = (vol - vmin) / (vmax - vmin)
    else:
        vol[:] = 0.0
    return vol

def to_uint8_stack(volume):
    vol = safe_minmax_norm(volume)
    return (vol * 255).astype(np.uint8)

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs axon channel
image  = img_ch1       # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: axon/soma channel

# --------------------------
# 2) SMOOTHING (CH1)
# --------------------------
image_smooth = gaussian(image, sigma=1)

# --------------------------
# 3) BLOB DETECTION (CH1)
# --------------------------
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
# convert sigma->radius for LoG blobs (3D)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)
print(f"Detected {len(blobs)} lysosomes.")

# --------------------------
# 4) REGIONAL QUANTIFICATION (CH1) on a 3D GRID
# --------------------------
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.digitize(blobs[:, 0], z_bins) - 1
    y_idx = np.digitize(blobs[:, 1], y_bins) - 1
    x_idx = np.digitize(blobs[:, 2], x_bins) - 1
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

# --- per-blob table ---
blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 5) NAPARI VIEWER (we'll combine Ch1+Ch2 later)
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# Optional region ID anchors (without text; we set text later for compatibility)
region_centers = []
region_ids_list = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids_list.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)

region_pts = viewer.add_points(
    region_centers,
    size=6,
    face_color='black',
    name='Region Labels'
)
# Try setting text on older/newer napari safely
try:
    region_pts.properties = {'rid': np.array(region_ids_list)}
    region_pts.text = {'text': '{rid}', 'size': 8, 'color': 'white', 'anchor': 'center'}
except Exception:
    pass

# --------------------------
# 3B) AXON SEGMENTATION — Meijering + local thresholds
# --------------------------
ch2 = rescale_intensity(image_2.astype(np.float32), in_range='image', out_range=(0,1))
ch2 = gaussian(ch2, sigma=0.6, preserve_range=True)
resp = meijering(ch2, sigmas=np.linspace(0.6, 2.0, 8), black_ridges=False)

axon_mask = np.zeros_like(resp, dtype=bool)
for z in range(resp.shape[0]):
    R = resp[z]
    t = threshold_local(R, block_size=51, offset=-0.6*np.std(R))
    axon_mask[z] = R > t

axon_mask = remove_small_objects(axon_mask, min_size=8000, connectivity=5)
axon_mask = binary_opening(axon_mask, ball(2))
axon_mask = binary_closing(axon_mask, ball(2))
axon_skel = skeletonize_3d(axon_mask)

# --------------------------
# 4) REGIONAL QUANTIFICATION (CH2) on a 3D GRID
# --------------------------
num_bins_2 = (4, 4, 4)
z_bins_2 = np.linspace(0, image_2.shape[0], num_bins_2[0] + 1, dtype=int)
y_bins_2 = np.linspace(0, image_2.shape[1], num_bins_2[1] + 1, dtype=int)
x_bins_2 = np.linspace(0, image_2.shape[2], num_bins_2[2] + 1, dtype=int)

region_records = []
for zi in range(num_bins_2[0]):
    for yi in range(num_bins_2[1]):
        for xi in range(num_bins_2[2]):
            z0, z1 = z_bins_2[zi], z_bins_2[zi+1]
            y0, y1 = y_bins_2[yi], y_bins_2[yi+1]
            x0, x1 = x_bins_2[xi], x_bins_2[xi+1]
            rid = zi * (num_bins_2[1] * num_bins_2[2]) + yi * num_bins_2[2] + xi

            sub_mask  = axon_mask[z0:z1, y0:y1, x0:x1]
            sub_skel  = axon_skel[z0:z1, y0:y1, x0:x1]
            vox_total = sub_mask.size
            vox_axon  = int(sub_mask.sum())
            if vox_axon == 0:
                continue  # only keep regions with axon voxels > 0
            vox_skel  = int(sub_skel.sum())

            region_records.append({
                "region_id": rid,
                "axon_voxels": vox_axon,
                "region_voxels": vox_total,
                "axon_volume_fraction": vox_axon / vox_total if vox_total > 0 else 0.0,
                "skeleton_voxels": vox_skel
            })

df_axon_regions = pd.DataFrame(region_records)

# --------------------------
# 4B) COLOCALIZATION: ch1 blobs inside axon mask (CH2)
# --------------------------
def _in_bounds(z, y, x, shape):
    return (0 <= z < shape[0]) and (0 <= y < shape[1]) and (0 <= x < shape[2])

in_axon = []
for zc, yc, xc in (blobs[:, :3] if len(blobs) > 0 else []):
    zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
    if _in_bounds(zz, yy, xx, axon_mask.shape):
        in_axon.append(bool(axon_mask[zz, yy, xx]))
    else:
        in_axon.append(False)

if len(blobs) > 0:
    df["in_axon_mask_ch2"] = in_axon
    coloc_summary = df.groupby("region_id").agg(
        blobs_total=("id", "size"),
        blobs_in_axon=("in_axon_mask_ch2", "sum")
    ).reset_index()
    coloc_summary["frac_blobs_in_axon"] = (
        coloc_summary["blobs_in_axon"] / coloc_summary["blobs_total"].replace(0, np.nan)
    )
else:
    coloc_summary = pd.DataFrame(columns=["region_id", "blobs_total", "blobs_in_axon", "frac_blobs_in_axon"])

df_axon_regions.to_csv("axon_regions_ch2.csv", index=False)
coloc_summary.to_csv("blobs_in_axon_by_region.csv", index=False)
print("Saved: axon_regions_ch2.csv and blobs_in_axon_by_region.csv")

# --------------------------
# 5) COMBINED NAPARI VIEWER (Ch1 + Ch2 together)
# --------------------------
# Axon mask as labels (set colors AFTER creation)
labels_layer = viewer.add_labels(
    axon_mask.astype(np.uint8),
    name='Axon mask (Ch2)',
    opacity=0.35,
)
try:
    labels_layer.color = {1: (1.0, 0.0, 1.0, 1.0)}  # magenta RGBA
except Exception:
    pass
labels_layer.blending = 'translucent_no_depth'

# Optional skeleton overlay
viewer.add_image(
    axon_skel.astype(np.uint8),
    name='Axon skeleton',
    blending='additive',
    contrast_limits=(0, 1),
    colormap='gray',
    opacity=0.8
)

# Lysosome points color-coded by colocalization (yellow=in-axon, cyan=outside)
in_mask_np = np.array(in_axon, dtype=bool) if len(blobs) > 0 else np.array([], dtype=bool)
colors = np.zeros((len(blobs), 4), dtype=float) if len(blobs) > 0 else np.zeros((0, 4), dtype=float)
if len(blobs) > 0:
    colors[in_mask_np]  = [1.0, 1.0, 0.0, 1.0]  # yellow
    colors[~in_mask_np] = [0.0, 1.0, 1.0, 1.0]  # cyan

points_layer = viewer.add_points(
    blobs[:, :3] if len(blobs) > 0 else np.empty((0,3)),
    size=(np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2),
    face_color=colors,
    name='Lysosomes (yellow=in axon)'
)
# Older napari: set edge props after creation
try:
    points_layer.edge_color = 'black'
    points_layer.edge_width = 0.3
except Exception:
    pass

# --------------------------
# 6) MOVIES / OVERLAYS
# --------------------------
# CH1 video (raw)
img_norm = to_uint8_stack(image)
frames = [img_norm[z] for z in range(img_norm.shape[0])]
try:
    imageio.mimsave('lysosome_stack.mp4', frames, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack.mp4')
except TypeError:
    imageio.mimsave('lysosome_stack.gif', frames, fps=8)
    print('Saved: lysosome_stack.gif')

# CH1 video with blobs overlay (colored by in-axon status)
frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame = img_norm[z].copy()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5] if len(blobs) > 0 else []
    for b in z_blobs:
        y, x = int(round(b[1])), int(round(b[2]))
        radius = int(round(b[3]))
        in_here = bool(axon_mask[z, y, x]) if (0 <= y < axon_mask.shape[1] and 0 <= x < axon_mask.shape[2]) else False
        color = (0, 255, 255) if in_here else (255, 255, 0)  # BGR
        cv2.circle(frame_rgb, (x, y), max(2, radius), color, 2)
    frames_with_blobs.append(frame_rgb)
try:
    imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack_blobs.mp4')
except TypeError:
    imageio.mimsave('lysosome_stack_blobs.gif', frames_with_blobs, fps=8)
    print('Saved: lysosome_stack_blobs.gif')

# CH2 base stack (raw)
img_norm_2 = to_uint8_stack(image_2)
frames_2 = [img_norm_2[z2] for z2 in range(img_norm_2.shape[0])]
try:
    imageio.mimsave('axon_stack_ch2.mp4', frames_2, fps=8, format='FFMPEG')
    print("Saved: axon_stack_ch2.mp4")
except TypeError:
    imageio.mimsave('axon_stack_ch2.gif', frames_2, fps=8)
    print("Saved: axon_stack_ch2.gif")

# CH2 overlay with axon mask (JET)
frames_mask_overlay = []
for z2 in range(img_norm_2.shape[0]):
    base = cv2.cvtColor(img_norm_2[z2], cv2.COLOR_GRAY2BGR)
    mask = (axon_mask[z2].astype(np.uint8) * 255)
    color_mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(base, 1.0, color_mask, 0.35, 0.0)
    frames_mask_overlay.append(overlay)
try:
    imageio.mimsave('axon_mask_overlay_ch2.mp4', frames_mask_overlay, fps=8, format='FFMPEG')
    print("Saved: axon_mask_overlay_ch2.mp4")
except TypeError:
    imageio.mimsave('axon_mask_overlay_ch2.gif', frames_mask_overlay, fps=8)
    print("Saved: axon_mask_overlay_ch2.gif")

# FUSED OVERLAY VIDEO (Ch1+Ch2 with axon mask + lysosomes)
frames_fused = []
for z in range(img_norm_2.shape[0]):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    # Axon mask in magenta (R+B)
    mask = (axon_mask[z].astype(np.uint8) * 255)
    magenta = np.zeros_like(base)
    magenta[..., 2] = mask  # R
    magenta[..., 0] = np.maximum(magenta[..., 0], mask)  # B
    overlay = cv2.addWeighted(base, 1.0, magenta, 0.35, 0.0)

    # Lysosome circles
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5] if len(blobs) > 0 else []
    for b in z_blobs:
        y, x = int(round(b[1])), int(round(b[2]))
        r = max(2, int(round(b[3])))
        in_here = bool(axon_mask[z, y, x]) if (0 <= y < axon_mask.shape[1] and 0 <= x < axon_mask.shape[2]) else False
        color = (0, 255, 255) if in_here else (255, 255, 0)  # BGR
        cv2.circle(overlay, (x, y), r, color, 2)
    frames_fused.append(overlay)

try:
    imageio.mimsave('ch1_ch2_fused.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch1_ch2_fused.mp4")
except TypeError:
    imageio.mimsave('ch1_ch2_fused.gif', frames_fused, fps=8)
    print("Saved: ch1_ch2_fused.gif")

# --------------------------
# 7) RUN VIEWER
# --------------------------
napari.run()

In [ ]:
#####VERSION 3######################################3

In [ ]:
# %%
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, meijering, threshold_local
import napari
import pandas as pd
import imageio
import cv2

from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, skeletonize_3d
)

# --------------------------
# Helpers
# --------------------------
def safe_minmax_norm(volume):
    vol = volume.astype(np.float32)
    vmin, vmax = float(vol.min()), float(vol.max())
    if vmax > vmin:
        vol = (vol - vmin) / (vmax - vmin)
    else:
        vol[:] = 0.0
    return vol

def to_uint8_stack(volume):
    vol = safe_minmax_norm(volume)
    return (vol * 255).astype(np.uint8)

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs axon channel
image  = img_ch1       # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: axon/soma channel

# --------------------------
# 2) SMOOTHING (CH1)
# --------------------------
image_smooth = gaussian(image, sigma=1)

# --------------------------
# 3) BLOB DETECTION (CH1)
# --------------------------
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
# convert sigma->radius for LoG blobs (3D)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)
print(f"Detected {len(blobs)} lysosomes.")

# --------------------------
# 4) REGIONAL QUANTIFICATION (CH1) on a 3D GRID
# --------------------------
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.digitize(blobs[:, 0], z_bins) - 1
    y_idx = np.digitize(blobs[:, 1], y_bins) - 1
    x_idx = np.digitize(blobs[:, 2], x_bins) - 1
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

# --- per-blob table ---
blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 5) NAPARI VIEWER (we'll combine Ch1+Ch2 later)
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# Optional region ID anchors (without text; we set text later for compatibility)
region_centers = []
region_ids_list = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids_list.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)

region_pts = viewer.add_points(
    region_centers,
    size=6,
    face_color='black',
    name='Region Labels'
)
# Try setting text (compatible with older/newer napari)
try:
    region_pts.properties = {'rid': np.array(region_ids_list)}
    region_pts.text = {'text': '{rid}', 'size': 8, 'color': 'white', 'anchor': 'center'}
except Exception:
    pass

# --------------------------
# 3B) AXON SEGMENTATION — Meijering + local thresholds
# --------------------------
ch2 = rescale_intensity(image_2.astype(np.float32), in_range='image', out_range=(0,1))
ch2 = gaussian(ch2, sigma=0.6, preserve_range=True)
resp = meijering(ch2, sigmas=np.linspace(0.6, 2.0, 8), black_ridges=False)

axon_mask = np.zeros_like(resp, dtype=bool)
for z in range(resp.shape[0]):
    R = resp[z]
    t = threshold_local(R, block_size=51, offset=-0.6*np.std(R))
    axon_mask[z] = R > t

axon_mask = remove_small_objects(axon_mask, min_size=8000, connectivity=5)
axon_mask = binary_opening(axon_mask, ball(2))
axon_mask = binary_closing(axon_mask, ball(2))
axon_skel = skeletonize_3d(axon_mask)

# --------------------------
# 3C) ASSIGN UNIQUE ID TO *EVERY* AXON VOXEL
# --------------------------
# axon_voxel_id: 0 = background; 1..N for each True voxel in axon_mask (C-order)
axon_voxel_id = np.zeros_like(axon_mask, dtype=np.int64)
n_ax_vox = int(axon_mask.sum())
if n_ax_vox > 0:
    axon_voxel_id[axon_mask] = np.arange(1, n_ax_vox + 1, dtype=np.int64)
print(f"Assigned IDs to {n_ax_vox} axon voxels.")

# Also keep a lookup table of voxel coordinates for each id
# coords: N x 3 array of (z,y,x) for True voxels
coords = np.argwhere(axon_mask)  # order matches boolean indexing iteration (C-order)
ids_at_coords = axon_voxel_id[axon_mask]  # 1..N in the same order
df_ax_voxel_index = pd.DataFrame({
    "axon_voxel_id": ids_at_coords,
    "z": coords[:, 0],
    "y": coords[:, 1],
    "x": coords[:, 2],
})
df_ax_voxel_index.to_csv("axon_voxel_index.csv", index=False)

# --------------------------
# 4) REGIONAL QUANTIFICATION (CH2) on a 3D GRID (kept from before)
# --------------------------
num_bins_2 = (4, 4, 4)
z_bins_2 = np.linspace(0, image_2.shape[0], num_bins_2[0] + 1, dtype=int)
y_bins_2 = np.linspace(0, image_2.shape[1], num_bins_2[1] + 1, dtype=int)
x_bins_2 = np.linspace(0, image_2.shape[2], num_bins_2[2] + 1, dtype=int)

region_records = []
for zi in range(num_bins_2[0]):
    for yi in range(num_bins_2[1]):
        for xi in range(num_bins_2[2]):
            z0, z1 = z_bins_2[zi], z_bins_2[zi+1]
            y0, y1 = y_bins_2[yi], y_bins_2[yi+1]
            x0, x1 = x_bins_2[xi], x_bins_2[xi+1]
            rid = zi * (num_bins_2[1] * num_bins_2[2]) + yi * num_bins_2[2] + xi

            sub_mask  = axon_mask[z0:z1, y0:y1, x0:x1]
            sub_skel  = axon_skel[z0:z1, y0:y1, x0:x1]
            vox_total = sub_mask.size
            vox_axon  = int(sub_mask.sum())
            if vox_axon == 0:
                continue
            vox_skel  = int(sub_skel.sum())

            region_records.append({
                "region_id": rid,
                "axon_voxels": vox_axon,
                "region_voxels": vox_total,
                "axon_volume_fraction": vox_axon / vox_total if vox_total > 0 else 0.0,
                "skeleton_voxels": vox_skel
            })

df_axon_regions = pd.DataFrame(region_records)

# --------------------------
# 4B) MAP LYSOSOMES TO AXON VOXEL IDs + COUNTS
# --------------------------
def _in_bounds(z, y, x, shape):
    return (0 <= z < shape[0]) and (0 <= y < shape[1]) and (0 <= x < shape[2])

lys_ax_voxel_ids = []
in_axon = []
if len(blobs) > 0:
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if _in_bounds(zz, yy, xx, axon_mask.shape):
            voxel_id = int(axon_voxel_id[zz, yy, xx])
            lys_ax_voxel_ids.append(voxel_id)
            in_axon.append(voxel_id > 0)
        else:
            lys_ax_voxel_ids.append(0)
            in_axon.append(False)
else:
    lys_ax_voxel_ids = []

if len(df) > 0:
    df["in_axon_mask_ch2"] = in_axon
    df["axon_voxel_id"] = lys_ax_voxel_ids
else:
    df["in_axon_mask_ch2"] = []
    df["axon_voxel_id"] = []

# per-voxel lysosome counts (ignore id=0 / outside axon)
if len(df) > 0:
    lys_counts = (df[df["axon_voxel_id"] > 0]
                  .groupby("axon_voxel_id")
                  .size()
                  .reset_index(name="lysosome_count"))
else:
    lys_counts = pd.DataFrame(columns=["axon_voxel_id", "lysosome_count"])

# Save detailed + counts tables
df.to_csv("lysosomes_with_axon_voxel_id.csv", index=False)
lys_counts.to_csv("lysosome_counts_per_axon_voxel.csv", index=False)

# Keep region-wise coloc summary as before (optional)
if len(df) > 0:
    coloc_summary = df.groupby("region_id").agg(
        blobs_total=("id", "size"),
        blobs_in_axon=("in_axon_mask_ch2", "sum")
    ).reset_index()
    coloc_summary["frac_blobs_in_axon"] = (
        coloc_summary["blobs_in_axon"] / coloc_summary["blobs_total"].replace(0, np.nan)
    )
else:
    coloc_summary = pd.DataFrame(columns=["region_id", "blobs_total", "blobs_in_axon", "frac_blobs_in_axon"])

df_axon_regions.to_csv("axon_regions_ch2.csv", index=False)
coloc_summary.to_csv("blobs_in_axon_by_region.csv", index=False)
print("Saved: axon_regions_ch2.csv, blobs_in_axon_by_region.csv, "
      "lysosomes_with_axon_voxel_id.csv, lysosome_counts_per_axon_voxel.csv, axon_voxel_index.csv")

# --------------------------
# 5) COMBINED NAPARI VIEWER (Ch1 + Ch2 together)
# --------------------------
# Axon mask as labels (binary layer)
labels_layer = viewer.add_labels(
    axon_mask.astype(np.uint8),
    name='Axon mask (Ch2)',
    opacity=0.35,
)
try:
    labels_layer.color = {1: (1.0, 0.0, 1.0, 1.0)}  # magenta
except Exception:
    pass
labels_layer.blending = 'translucent_no_depth'

# OPTIONAL: Per-voxel ID labels volume (can be heavy → hidden by default)
labels_ids = viewer.add_labels(
    axon_voxel_id.astype(np.int32),
    name='Axon voxel IDs (per-voxel)',
    opacity=0.25,
    visible=False  # toggle on if needed
)

# Optional skeleton overlay
viewer.add_image(
    axon_skel.astype(np.uint8),
    name='Axon skeleton',
    blending='additive',
    contrast_limits=(0, 1),
    colormap='gray',
    opacity=0.8
)

# Lysosome points with colors + TEXT showing mapped axon voxel ID
in_mask_np = np.array(in_axon, dtype=bool) if len(blobs) > 0 else np.array([], dtype=bool)
colors = np.zeros((len(blobs), 4), dtype=float) if len(blobs) > 0 else np.zeros((0, 4), dtype=float)
if len(blobs) > 0:
    colors[in_mask_np]  = [1.0, 1.0, 0.0, 1.0]  # yellow (inside)
    colors[~in_mask_np] = [0.0, 1.0, 1.0, 1.0]  # cyan (outside)

points_layer = viewer.add_points(
    blobs[:, :3] if len(blobs) > 0 else np.empty((0,3)),
    size=(np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2),
    face_color=colors,
    name='Lysosomes'
)
# Edge props (older napari: set after)
try:
    points_layer.edge_color = 'black'
    points_layer.edge_width = 0.3
except Exception:
    pass
# Add per-lysosome properties & text label
try:
    if len(blobs) > 0:
        points_layer.properties = {
            'lys_id': df['id'].to_numpy(),
            'axon_voxel_id': df['axon_voxel_id'].to_numpy()
        }
        # Show the voxel ID for each lysosome; you can change text to '{lys_id}' or 'L{lys_id}|V{axon_voxel_id}'
        points_layer.text = {'text': '{axon_voxel_id}', 'size': 8, 'color': 'white', 'anchor': 'upper left'}
except Exception:
    pass

# OPTIONAL (heavy): add a point for EVERY axon voxel with text=ID (hidden by default)
try:
    if n_ax_vox > 0:
        ax_vox_pts = viewer.add_points(
            coords,                    # (z, y, x)
            size=2,
            face_color='magenta',
            name='Axon voxel ID points',
            visible=False              # toggle on if needed
        )
        ax_vox_pts.properties = {'axon_voxel_id': ids_at_coords}
        ax_vox_pts.text = {'text': '{axon_voxel_id}', 'size': 6, 'color': 'magenta', 'anchor': 'center'}
except Exception:
    pass

# --------------------------
# 6) MOVIES / OVERLAYS (unchanged)
# --------------------------
# CH1 video (raw)
img_norm = to_uint8_stack(image)
frames = [img_norm[z] for z in range(img_norm.shape[0])]
try:
    imageio.mimsave('lysosome_stack.mp4', frames, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack.mp4')
except TypeError:
    imageio.mimsave('lysosome_stack.gif', frames, fps=8)
    print('Saved: lysosome_stack.gif')

# CH1 video with blobs overlay (colored by in-axon status)
frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame = img_norm[z].copy()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5] if len(blobs) > 0 else []
    for b in z_blobs:
        y, x = int(round(b[1])), int(round(b[2]))
        radius = int(round(b[3]))
        in_here = bool(axon_mask[z, y, x]) if (0 <= y < axon_mask.shape[1] and 0 <= x < axon_mask.shape[2]) else False
        color = (0, 255, 255) if in_here else (255, 255, 0)  # BGR
        cv2.circle(frame_rgb, (x, y), max(2, radius), color, 2)
    frames_with_blobs.append(frame_rgb)
try:
    imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack_blobs.mp4')
except TypeError:
    imageio.mimsave('lysosome_stack_blobs.gif', frames_with_blobs, fps=8)
    print('Saved: lysosome_stack_blobs.gif')

# CH2 base stack (raw)
img_norm_2 = to_uint8_stack(image_2)
frames_2 = [img_norm_2[z2] for z2 in range(img_norm_2.shape[0])]
try:
    imageio.mimsave('axon_stack_ch2.mp4', frames_2, fps=8, format='FFMPEG')
    print("Saved: axon_stack_ch2.mp4")
except TypeError:
    imageio.mimsave('axon_stack_ch2.gif', frames_2, fps=8)
    print("Saved: axon_stack_ch2.gif")

# CH2 overlay with axon mask (JET)
frames_mask_overlay = []
for z2 in range(img_norm_2.shape[0]):
    base = cv2.cvtColor(img_norm_2[z2], cv2.COLOR_GRAY2BGR)
    mask = (axon_mask[z2].astype(np.uint8) * 255)
    color_mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(base, 1.0, color_mask, 0.35, 0.0)
    frames_mask_overlay.append(overlay)
try:
    imageio.mimsave('axon_mask_overlay_ch2.mp4', frames_mask_overlay, fps=8, format='FFMPEG')
    print("Saved: axon_mask_overlay_ch2.mp4")
except TypeError:
    imageio.mimsave('axon_mask_overlay_ch2.gif', frames_mask_overlay, fps=8)
    print("Saved: axon_mask_overlay_ch2.gif")

# FUSED OVERLAY VIDEO (Ch1+Ch2 with axon mask + lysosomes)
frames_fused = []
for z in range(img_norm_2.shape[0]):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    # Axon mask in magenta (R+B)
    mask = (axon_mask[z].astype(np.uint8) * 255)
    magenta = np.zeros_like(base)
    magenta[..., 2] = mask  # R
    magenta[..., 0] = np.maximum(magenta[..., 0], mask)  # B
    overlay = cv2.addWeighted(base, 1.0, magenta, 0.35, 0.0)

    # Lysosome circles
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5] if len(blobs) > 0 else []
    for b in z_blobs:
        y, x = int(round(b[1])), int(round(b[2]))
        r = max(2, int(round(b[3])))
        in_here = bool(axon_mask[z, y, x]) if (0 <= y < axon_mask.shape[1] and 0 <= x < axon_mask.shape[2]) else False
        color = (0, 255, 255) if in_here else (255, 255, 0)  # BGR
        cv2.circle(overlay, (x, y), r, color, 2)
    frames_fused.append(overlay)

try:
    imageio.mimsave('ch1_ch2_fused.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch1_ch2_fused.mp4")
except TypeError:
    imageio.mimsave('ch1_ch2_fused.gif', frames_fused, fps=8)
    print("Saved: ch1_ch2_fused.gif")

# --------------------------
# 7) RUN VIEWER
# --------------------------
napari.run()